# L5 — Data Wrangling con Pandas (Bank Marketing – UCI)

## 📍 Situación inicial
Una empresa fintech recibe datos desde múltiples fuentes; los datos llegan desordenados, con nulos y duplicados, lo que afecta la precisión de los reportes. Se requiere un proceso de **Data Wrangling con Pandas** para limpiar, transformar e integrar los datos en un único DataFrame de calidad. :contentReference[oaicite:0]{index=0}

## 🎯 Objetivo
Aplicar un flujo reproducible de **carga, limpieza, transformación y optimización** de datos con Pandas para dejar un dataset listo para análisis y reportes. :contentReference[oaicite:1]{index=1}1)

## 🗂️ Dataset
Trabajaremos con **Bank Marketing (UCI)**, que contiene variables demográficas y de contacto de clientes bancarios. (CSV único para todo el caso).

## 🛠️ Instrucciones
1. **Carga y exploración de datos**  
   - Importa el CSV en un DataFrame.  
   - Explora con `.head()`, `.info()` y `.describe()`.  
   - Identifica **valores nulos** y **duplicados**. :contentReference[oaicite:2]{index=2}
2. **Limpieza y transformación de datos**  
   - Imputa nulos con una estrategia adecuada (**media**, **mediana** o **moda**).  
   - Elimina registros **duplicados**.  
   - Convierte categóricas a numéricas si es necesario (p. ej., *yes/no* → 1/0). :contentReference[oaicite:3]{index=3}
3. **Optimización y estructuración**  
   - Aplica `groupby` + **agregaciones**.  
   - Realiza **filtros** para obtener subconjuntos de interés.  
   - **Renombra** y **reorganiza** columnas para mejorar la interpretación. :contentReference[oaicite:4]{index=4}
4. **Exportación**  
   - Guarda el DataFrame procesado en **CSV** (sin índice).  
   - Exporta los datos limpios a **Excel**. :contentReference[oaicite:5]{index=5}

## 📬 Entregables
1) **Código en Python** con la implementación.  
2) **Explicación técnica** paso a paso y justificación.  
3) **Ejemplo antes vs. después** de la transformación.  
4) **Conclusiones** sobre la importancia del Data Wrangling en la calidad de los datos. :contentReference[oaicite:6]{index=6}


1) Carga y exploración para pegar como una sola celda en Colab (dataset: Bank Marketing – UCI).

In [4]:
# 1) Carga y exploración — Bank Marketing (UCI) (bank-additional-full)
import pandas as pd

URL = "https://raw.githubusercontent.com/alexkataev/Case-Study-UCI-Bank-Marketing-Dataset/refs/heads/main/data/bank-additional-full.csv"
df = pd.read_csv(URL, sep=';', low_memory=False)
print("✅ CSV cargado | shape:", df.shape)

# Exploración rápida
print("\n== head(5) ==")
print(df.head())

print("\n== info() ==")
print(df.info())

print("\n== describe(numérico) ==")
num_desc = df.select_dtypes(include='number').describe().T
print(num_desc)

# Nulos reales (NaN)
print("\n== Nulos por columna (NaN) ==")
print(df.isna().sum().sort_values(ascending=False).head(20))

# 'unknown' en categóricas (nulos semánticos)
obj_cols = df.select_dtypes(include='object').columns
unknown_counts = df[obj_cols].apply(lambda s: s.str.lower().eq('unknown').sum())
print("\n== Conteo de 'unknown' por columna categórica ==")
print(unknown_counts.sort_values(ascending=False).head(20))

# Duplicados
print("\n== Total de filas duplicadas ==")
print(df.duplicated().sum())


✅ CSV cargado | shape: (41188, 21)

== head(5) ==
   age        job  marital    education  default housing loan    contact  \
0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   57   services  married  high.school  unknown      no   no  telephone   
2   37   services  married  high.school       no     yes   no  telephone   
3   40     admin.  married     basic.6y       no      no   no  telephone   
4   56   services  married  high.school       no      no  yes  telephone   

  month day_of_week  ...  campaign  pdays  previous     poutcome emp.var.rate  \
0   may         mon  ...         1    999         0  nonexistent          1.1   
1   may         mon  ...         1    999         0  nonexistent          1.1   
2   may         mon  ...         1    999         0  nonexistent          1.1   
3   may         mon  ...         1    999         0  nonexistent          1.1   
4   may         mon  ...         1    999         0  nonexistent          1.1   

   con

2) Limpieza y transformación — Bank Marketing (UCI)

In [5]:
# 2) Limpieza y transformación — Bank Marketing (UCI)
import pandas as pd

print("== 2) Limpieza y transformación ==")
df_clean = df.copy()

# 2.1 Imputar 'unknown' en columnas categóricas con la moda de cada columna
obj_cols = df_clean.select_dtypes(include='object').columns
before_unknown = df_clean[obj_cols].apply(lambda s: (s.str.lower() == 'unknown').sum())
print("\n[2.1] 'unknown' antes (top):")
print(before_unknown.sort_values(ascending=False).head(10))

for col in obj_cols:
    mask_unknown = df_clean[col].str.lower().eq('unknown')
    if mask_unknown.any():
        moda = df_clean.loc[~mask_unknown, col].mode()
        if not moda.empty:
            df_clean.loc[mask_unknown, col] = moda[0]

after_unknown = df_clean[obj_cols].apply(lambda s: (s.str.lower() == 'unknown').sum())
print("\n[2.1] 'unknown' después (debería ser 0 en las columnas imputadas):")
print(after_unknown.sort_values(ascending=False).head(10))

# 2.2 Eliminar duplicados (fila completa)
dup_count = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
print(f"\n[2.2] Duplicados eliminados: {dup_count}")

# 2.3 Convertir yes/no -> 1/0 en columnas binarias clave
yn_cols = [c for c in ['default', 'housing', 'loan', 'y'] if c in df_clean.columns]
for c in yn_cols:
    df_clean[c] = df_clean[c].map({'yes': 1, 'no': 0}).astype('Int64')

print("\n[2.3] Mapeo yes/no aplicado en:", yn_cols)

# 2.4 Verificación de tipos numéricos en columnas principales
num_cols = [c for c in ['age','duration','campaign','pdays','previous',
                        'emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed']
            if c in df_clean.columns]
for c in num_cols:
    df_clean[c] = pd.to_numeric(df_clean[c], errors='coerce')

print("\n[2.4] dtypes (muestra):")
print(df_clean.dtypes.loc[yn_cols + num_cols].head(20))

print("\nShape final:", df_clean.shape)


== 2) Limpieza y transformación ==

[2.1] 'unknown' antes (top):
default        8597
education      1731
housing         990
loan            990
job             330
marital          80
contact           0
month             0
day_of_week       0
poutcome          0
dtype: int64

[2.1] 'unknown' después (debería ser 0 en las columnas imputadas):
job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
poutcome       0
dtype: int64

[2.2] Duplicados eliminados: 14

[2.3] Mapeo yes/no aplicado en: ['default', 'housing', 'loan', 'y']

[2.4] dtypes (muestra):
default             Int64
housing             Int64
loan                Int64
y                   Int64
age                 int64
duration            int64
campaign            int64
pdays               int64
previous            int64
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.emp

3) Optimización y estructuración

In [6]:
# 3) Optimización y estructuración
import pandas as pd

print("== 3) Optimización y estructuración ==")

# 3.1 Selección de columnas y renombrado a español
cols = [
    'y','age','job','marital','education','default','housing','loan',
    'contact','month','day_of_week','duration','campaign','pdays','previous','poutcome',
    'emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed'
]
cols = [c for c in cols if c in df_clean.columns]

rename_map = {
    'y':'suscrito', 'age':'edad', 'job':'trabajo', 'marital':'estado_civil',
    'education':'educacion', 'default':'en_mora', 'housing':'hipoteca', 'loan':'prestamo',
    'contact':'contacto', 'month':'mes', 'day_of_week':'dia_semana',
    'duration':'duracion', 'campaign':'campana', 'pdays':'p_dias',
    'previous':'previas', 'poutcome':'resultado_prev',
    'emp.var.rate':'tasa_var_empleo', 'cons.price.idx':'indice_precio',
    'cons.conf.idx':'indice_confianza', 'euribor3m':'euribor3m',
    'nr.employed':'empleados'
}

df_ready = df_clean[cols].rename(columns=rename_map).copy()

# Orden de columnas
orden = [
    'suscrito','edad','trabajo','educacion','estado_civil','hipoteca','prestamo','en_mora',
    'contacto','mes','dia_semana','duracion','campana','p_dias','previas','resultado_prev',
    'tasa_var_empleo','indice_precio','indice_confianza','euribor3m','empleados'
]
df_ready = df_ready[[c for c in orden if c in df_ready.columns]]

print("\n[3.1] Vista (top 5) post renombrado/reorden:")
print(df_ready.head())

# 3.2 Segmentación: tasa de suscripción por trabajo x educación
if {'trabajo','educacion','suscrito'}.issubset(df_ready.columns):
    df_ready['suscrito'] = df_ready['suscrito'].astype('Int64')
    seg = (
        df_ready
        .groupby(['trabajo','educacion'], dropna=False)
        .agg(
            n=('suscrito','size'),
            edad_prom=('edad','mean'),
            duracion_med=('duracion','median'),
            tasa_suscripcion=('suscrito','mean')
        )
        .reset_index()
    )
    seg['edad_prom'] = seg['edad_prom'].round(1)
    seg['duracion_med'] = seg['duracion_med'].round(0)
    seg['tasa_suscripcion'] = (seg['tasa_suscripcion']*100).round(2)

    print("\n[3.2] Segmentos trabajo x educación (Top por tasa de suscripción %):")
    print(seg.sort_values('tasa_suscripcion', ascending=False).head(10))
else:
    print("\n[3.2] No están todas las columnas necesarias para el groupby de segmentos.")

# 3.3 Filtros de interés
muestras = []

if {'edad','hipoteca'}.issubset(df_ready.columns):
    f1 = df_ready[(df_ready['edad'] >= 60) & (df_ready['hipoteca'] == 1)][
        ['edad','trabajo','educacion','hipoteca','suscrito']
    ].head(10)
    muestras.append(("Mayores de 60 con hipoteca=1", f1))

if {'duracion'}.issubset(df_ready.columns):
    f2 = df_ready[df_ready['duracion'] > 300][
        ['duracion','trabajo','educacion','suscrito']
    ].head(10)
    muestras.append(("Llamadas con duración > 300s", f2))

if {'previas','resultado_prev'}.issubset(df_ready.columns):
    f3 = df_ready[(df_ready['previas'] > 0) & (df_ready['resultado_prev'].str.lower() != 'nonexistent')][
        ['previas','resultado_prev','trabajo','suscrito']
    ].head(10)
    muestras.append(("Clientes con contactos previos efectivos", f3))

for titulo, dfm in muestras:
    print(f"\n[3.3] {titulo} (muestra):")
    print(dfm)

# Resultado para exportación en el punto 4
df_final = df_ready.copy()
print("\nShape df_final (listo para exportar):", df_final.shape)


== 3) Optimización y estructuración ==

[3.1] Vista (top 5) post renombrado/reorden:
   suscrito  edad    trabajo    educacion estado_civil  hipoteca  prestamo  \
0         0    56  housemaid     basic.4y      married         0         0   
1         0    57   services  high.school      married         0         0   
2         0    37   services  high.school      married         1         0   
3         0    40     admin.     basic.6y      married         0         0   
4         0    56   services  high.school      married         0         1   

   en_mora   contacto  mes  ... duracion  campana  p_dias  previas  \
0        0  telephone  may  ...      261        1     999        0   
1        0  telephone  may  ...      149        1     999        0   
2        0  telephone  may  ...      226        1     999        0   
3        0  telephone  may  ...      151        1     999        0   
4        0  telephone  may  ...      307        1     999        0   

   resultado_prev tasa_va

4) Exportación de datos — CSV y Excel

In [7]:
# 4) Exportación de datos — CSV y Excel

# Rutas de salida (en la misma carpeta del notebook)
csv_path = "bank_marketing_limpio.csv"
xlsx_path = "bank_marketing_limpio.xlsx"

# Exportar
df_final.to_csv(csv_path, index=False)
df_final.to_excel(xlsx_path, index=False)

print("✅ Exportación completa")
print("CSV  ->", csv_path)
print("XLSX ->", xlsx_path)

# Verificación rápida
print("\nVista rápida del CSV exportado:")
print(pd.read_csv(csv_path).head())


✅ Exportación completa
CSV  -> bank_marketing_limpio.csv
XLSX -> bank_marketing_limpio.xlsx

Vista rápida del CSV exportado:
   suscrito  edad    trabajo    educacion estado_civil  hipoteca  prestamo  \
0         0    56  housemaid     basic.4y      married         0         0   
1         0    57   services  high.school      married         0         0   
2         0    37   services  high.school      married         1         0   
3         0    40     admin.     basic.6y      married         0         0   
4         0    56   services  high.school      married         0         1   

   en_mora   contacto  mes  ... duracion  campana  p_dias  previas  \
0        0  telephone  may  ...      261        1     999        0   
1        0  telephone  may  ...      149        1     999        0   
2        0  telephone  may  ...      226        1     999        0   
3        0  telephone  may  ...      151        1     999        0   
4        0  telephone  may  ...      307        1     99